In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm $DL/*

In [3]:
from datetime import datetime
import os

import numpy as np
import pandas as pd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'AMD', 'AMZN', 'ANET', 'APPH', 'AXON',
    'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG', 'DKNG',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'FUV', 'FVRR', 'GH', 'GMED',
    'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU','MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW',
    'PINS', 'PTON', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE',
    'SHOP', 'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [7]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Holdings - Damian Satterthwaite-Phillips.csv
Portfolio_Positions_Apr-01-2022.csv
Positions.csv


In [8]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [9]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [10]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
AMD
AMZN


### E*Trade

In [11]:
DONGMEI = False

In [12]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
AAPL,6275.16
ABMD,2370.48
ABNB,1384.56
AMZN,3271.2
ANET,278.96


In [13]:
#et_df.loc[et_df.ET == '--']
#et_df.loc['TREX', 'ET'] = 68.96

In [14]:
check_symbols(et_df)

All clear


In [15]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,6275
ABMD,2370
ABNB,1385
AMD,0
AMZN,3271


### Fidelity

In [16]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Apr-01-2022.csv'

In [17]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,10846.03,$1.00,$0.00,$10846.03,NaN,NaN,NaN,NaN,22.03%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,16.00,$174.31,-$0.30,$2788.96,-$4.80,-0.18%,+$364.80,+15.04%,5.66%,$2424.16,$151.51,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,1.00,$338.64,+$7.40,$338.64,+$7.40,+2.23%,+$52.78,+18.46%,0.69%,$285.86,$285.86,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,3.00,$173.07,+$1.31,$519.21,+$3.93,+0.76%,+$90.96,+21.23%,1.05%,$428.25,$142.75,Cash
4,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,10.00,$108.19,-$1.15,$1081.90,-$11.50,-1.06%,+$56.56,+5.51%,2.20%,$1025.34,$102.53,Cash


In [18]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,2789
2,ROLLOVER IRA,ABMD,339
3,ROLLOVER IRA,ABNB,519
4,ROLLOVER IRA,AMD,1082
5,ROLLOVER IRA,ANET,1534


In [19]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,2789
ABMD,339
ABNB,519
AMD,1082
ANET,1534


In [20]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [21]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,6275,2789,3138,0
ABMD,2370,339,1355,0
ABNB,1385,519,1038,0
AMD,0,1082,6708,0
AMZN,3271,0,3271,0


In [22]:
out.to_csv(f'{DL}/clean.csv')

In [23]:
!open $DL/clean.csv

In [24]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Holdings - Damian Satterthwaite-Phillips.csv


### MarketWatch

In [28]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


In [29]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


,Value_0,Value_1,Value_2
AAPL,872,3138,2092
ABMD,677,1016,0
ABNB,1038,173,0
AMD,757,2056,1839
AMZN,3271,0,0


In [30]:
check_symbols(mw)

All clear


In [31]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_0,Value_1,Value_2
AAPL,872,3138,2092
ABMD,677,1016,0
ABNB,1038,173,0
AMD,757,2056,1839
AMZN,3271,0,0


In [32]:
mw.to_csv(f'{DL}/mw.csv')

In [33]:
!open $DL/mw.csv

In [19]:
!rm $DL/"Holdings - Damian Satterthwaite-Phillips.csv" $DL/mw.csv
!ls $DL

Portfolio_Positions_Mar-31-2022.csv Positions.csv


In [17]:
!rm $DL/*

rm: /Users/damiansp/Downloads/*: No such file or directory
